In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/SpamMailDetection/TrMail.csv')

In [ ]:
print("Veri kümesindeki satır sayısı:", df.shape[0])

Veri kümesindeki satır sayısı: 826


In [ ]:
df.head(5)

,Text,Classification,spam
0,Sayın Yetkili \n Gelişen ve değişen günümüz ...,spam,NaN
1,Sayın Yetkili \n \n28 Kasım 2010 tarihli KPSS ...,spam,NaN
2,Sayın Yetkili \n Gelişen ve değişen günümüz ...,spam,NaN
3,T.C.\nURLA KAYMAKAMLIĞI\nURLA HAKAN ÇEKEN ANAD...,spam,NaN
4,Sayın Yetkili Hızla büyüyen ve gelişmekte ola...,spam,NaN


In [ ]:
df = df.drop(columns=['spam'])
# Yeni DataFrame'i kontrol edin
df.head()

,Text,Classification
0,Sayın Yetkili \n Gelişen ve değişen günümüz ...,spam
1,Sayın Yetkili \n \n28 Kasım 2010 tarihli KPSS ...,spam
2,Sayın Yetkili \n Gelişen ve değişen günümüz ...,spam
3,T.C.\nURLA KAYMAKAMLIĞI\nURLA HAKAN ÇEKEN ANAD...,spam
4,Sayın Yetkili Hızla büyüyen ve gelişmekte ola...,spam


In [ ]:
#Aynı verileri sil
df.drop_duplicates(subset ="Text",keep = False, inplace = True)
# Satır sayısını öğrenmek için
satir_sayisi = df.shape[0]

# Satır sayısını yazdırın
print(f"Veri setindeki satır sayısı: {satir_sayisi}")

Veri setindeki satır sayısı: 713


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\<.*?\>', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['Text'] = df['Text'].apply(clean_text)


In [ ]:
X = df['Text']
y = df['Classification']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [ ]:
y_pred = model.predict(X_test_tfidf)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Classification Report
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.8881
              precision    recall  f1-score   support

    non-spam       0.85      1.00      0.92        91
        spam       1.00      0.69      0.82        52

    accuracy                           0.89       143
   macro avg       0.93      0.85      0.87       143
weighted avg       0.90      0.89      0.88       143



In [ ]:
import joblib

model_path = '/content/drive/My Drive/Colab Notebooks/SpamMailDetection/spam_classifier_model.pkl'
vectorizer_path = '/content/drive/My Drive/Colab Notebooks/SpamMailDetection/tfidf_vectorizer.pkl'

# Modeli kaydet
joblib.dump(model, model_path)

# Vectorizer'ı kaydet
joblib.dump(vectorizer, vectorizer_path)


['/content/drive/My Drive/Colab Notebooks/SpamMailDetection/tfidf_vectorizer.pkl']

In [ ]:
# Modeli yükle
model = joblib.load(model_path)

# Vectorizer'ı yükle
vectorizer = joblib.load(vectorizer_path)

In [ ]:
# Örnek yeni veri
new_texts = ["3175 Kodlu Önlisans Bankacılık ve Sigortacılık bölümü mezunları adsdfsdfsdfsdf ygs"]

# Veriyi temizle
cleaned_texts = [clean_text(text) for text in new_texts]

In [ ]:
# Yeni veriyi TF-IDF ile vektörleştir
new_texts_tfidf = vectorizer.transform(cleaned_texts)

In [ ]:
# Tahminleri al
predictions = model.predict(new_texts_tfidf)

# Sonuçları göster
for text, label in zip(new_texts, predictions):
    print(f"Metin: {text} -> Sınıflandırma: {'Spam' if label == 'spam' else 'Spam Değil'}")


Metin: 3175 Kodlu Önlisans Bankacılık ve Sigortacılık bölümü mezunları adsdfsdfsdfsdf ygs -> Sınıflandırma: Spam
